In [1]:
import matplotlib.pyplot as plt
import numpy as np

label_font = {'fontname':'Arial', 'size':'11', 'color':'black', 'weight':'normal',
          'verticalalignment':'bottom'} 
title_font = {'fontname':'sans-serif', 'size':'16', 'color':'red', 'weight':'bold'}
legend_font = {'family':'sans-serif', 'size':'10', 'style':'normal'} 


In [2]:
def genWhiteNoise(mu, sigma, size):
    """
    Generate samples of normally distributed white noise
    
    :param mu: Mean of the white noise
    :type mu: float
    :param sigma: Standard deviation of the noise 
    :type sigma: float
    :param size: Number of samples to generate
    :type size: integer
    :return: Array of normally distributed data
    :rtype: numpy array
    """
    samples = np.random.normal(mu, sigma, size)
    return samples


def genTimeVals(fs, size, timeUncertainty):
    """
    Generate time samples with jitter
    
    :param fs: Sampling frequency (in Hz)
    :type fs: float
    :param size: Number of samples to generate
    :type size: integer
    :return: Array of time samples
    :rtype: numpy array
    """
    uncertainties =  np.random.normal(0, timeUncertainty, size = size)
    timeVals = np.asarray(range(int(size))) / fs + uncertainties 
    return timeVals


def genSignal(amplitude, frequency, times):
    """
    Generate sinusoidal data

    :param amplitude: Amplitude
    :type amplitude: float
    :param frequency: Frequency of sine wave
    :type frequency: float
    :param time: Times to sample sine wave at
    :type time: numpy array
    :return: Array of sampled sine wave
    :rtype: numpy array
    """
    return amplitude*np.sin(2*np.pi*frequency*times)


def checkPowerOf10(value):
    """
    Check if a number is a power of 10
    
    :param value: Number to check
    :type value: float
    :return: Boolean if number is a power of 10
    :rtype: Boolean
    """
    value = int(value)
    while (value >= 10 and value % 10 == 0):
        value /= 10
    return value == 1

def convADCCount(dynamicRange, offset, bitDepth, data):
    stepSize = dynamicRange / 2**bitDepth
    # The below makes it so that an offset of 0 centers the data at 0
    lowVal = offset - dynamicRange / 2.
    highVal = offset + dynamicRange / 2.
    ADCVals = [int((x - lowVal)/stepSize) for x in data]
    ADCVals = [x if (x > 0 and x < 2**bitDepth) else (0 if x < 0 else 2**bitDepth - 1) for x in ADCVals]
    return np.asarray(ADCVals)



In [ ]:
sigFreq = 4e8
numSamps = 2**20

sampFreq = 1.E9
sigAmp = 0.001

muNoise = 0
sigNoise = 0.1

binWidth = sampFreq/numSamps
timeUncertainty = .1 * 1/sampFreq

timeValsPerfect = genTimeVals(sampFreq, numSamps, 0)
sigValsPerfect = genSignal(sigAmp, sigFreq, timeValsPerfect)

timeVals = genTimeVals(sampFreq, numSamps, timeUncertainty)
sigVals = genSignal(sigAmp, sigFreq, timeVals)

noiseVals = genWhiteNoise(muNoise, sigNoise, numSamps)

dataVals = noiseVals + sigVals
dataValsPerfect = noiseVals + sigValsPerfect

# Plot the data
if 0:
    #%matplotlib widget
    plt.close('all')
    #Scatter plot comparing jittered signal to perfectly sampled signal
    if 1:
        #I changed to plot sampled data against *perfect* time vals. System maps imperfect samples to grid it thinks is correct
        plt.figure()
        plt.scatter(timeValsPerfect*10**6, dataVals, label = 'jitter')
        plt.scatter(timeValsPerfect*10**6, dataValsPerfect, label = 'no jitter')

        # The weird symbol is the unicode character for mu
        plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
        plt.ylabel('Amplitude (V)', **label_font)
        plt.show()
        plt.legend()
        plt.title('Compare signal sampled with jitter to one without')

    if 1:
        plt.figure()
        plt.plot(timeValsPerfect*10**6, dataVals)

        # The weird symbol is the unicode character for mu
        plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
        plt.ylabel('Amplitude (V)', **label_font)
        plt.show()



In [ ]:
# dynamic range, offset, bit depth, data
dynamicRange = 1
offset = 0
bitDepth = 8
ADCVals = convADCCount(dynamicRange, offset, bitDepth, dataVals)
ADCValsVolts = ADCVals*1.0*dynamicRange/2**bitDepth + offset - dynamicRange/2

# Plot the data
if 0:
    plt.close('all')
    if 1:
        #sampled ADC counts vs time
        plt.figure()
        plt.plot(timeValsPerfect*10**6, ADCVals)

        # The weird symbol is the unicode character for mu
        plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
        plt.ylabel('Amplitude (ADC Counts)', **label_font)
        plt.show()
    if 1:
        # sampled voltage vs time
        plt.figure()
        plt.plot(timeValsPerfect*10**6, ADCValsVolts)

        # The weird symbol is the unicode character for mu
        plt.xlabel(u'Time (\u03BCs)', labelpad = 15, **label_font)
        plt.ylabel('Amplitude (V)', **label_font)
        plt.show()


In [ ]:
#%matplotlib widget
fftLen = len(ADCVals)
freqData = (np.abs(np.fft.fft(ADCValsVolts, fftLen))[0:int(fftLen/2)+1])**2
freqData = freqData/fftLen**2
freqData = np.asarray([2*x if (counter > 0 and counter < int(fftLen/2)) else x for counter, x in enumerate(freqData)])

freqStep = sampFreq/fftLen
frequencies = np.asarray(range(0, int(fftLen/2)+1))*freqStep

freqDataLog = 10*np.log10(freqData*1000/50)

maxFreq = frequencies[np.argmax(freqDataLog[1:]) + 1]/10**6
print('MAX FREQUENCY AT ' + str(round(maxFreq, 6)) + ' MHz')
print('MAX BIN AT ' + str(np.argmax(freqDataLog[1:]) + 1))

if 1:
    plt.figure()
    plt.plot((frequencies/10.**6)[1:], freqDataLog[1:])
    plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
    plt.ylabel(r'Power (dBm)', **label_font)
    plt.show()


In [ ]:
# Now do many averages of the data - model the frequency uncertainty as a change on the sampling frequency
plt.close('all')

sigFreq = 1.2345E8
numSamps = 2**20

sampFreq = 1.E9
sigAmp = 0.01

muNoise = 0
sigNoise = 0.1

numAvg = 100
#freqUncertainty = 1E3

dynamicRange = 1
offset = 0
bitDepth = 8

 
freqUncertainties = [1E0, 1E1, 1E2, 1E3, 1E4, 1E5, 1E6, 1E7]
maxAmps = []

for anUncertainty in freqUncertainties:
    freqDataTotal = np.zeros(int(numSamps/2)+1)
    print('ON UNCERTAINTY ' + str(anUncertainty))
    for x in range(numAvg):
        if x and x % 10 == 9:
            print('ON ITERATION '+ str(x + 1))
        timeVals = genTimeVals(sampFreq + np.random.normal(0, anUncertainty), numSamps, 0)
        sigVals = genSignal(sigAmp, sigFreq, timeVals)
        noiseVals = genWhiteNoise(muNoise, sigNoise, numSamps)
        dataVals = noiseVals + sigVals
        ADCVals = convADCCount(dynamicRange, offset, bitDepth, dataVals)
        ADCValsVolts = ADCVals*1.0*dynamicRange/2**bitDepth + offset - dynamicRange/2
        freqData = (np.abs(np.fft.fft(ADCValsVolts, numSamps))[0:int(numSamps/2)+1])**2
        freqData = freqData/numSamps**2
        freqData = np.asarray([2*x if (counter > 0 and counter < int(fftLen/2)) else x for counter, x in enumerate(freqData)])
        freqDataTotal += freqData


    freqStep = sampFreq/numSamps
    frequencies = np.asarray(range(0, int(numSamps/2)+1))*freqStep
    freqDataLog = 10*np.log10(freqDataTotal*1000/50/numAvg)
    maxFreq = frequencies[np.argmax(freqDataLog[1:]) + 1]/10**6

    print('MAX FREQUENCY AT ' + str(round(maxFreq, 6)) + ' MHz')
    print('MAX BIN AT ' + str(np.argmax(freqDataLog[1:]) + 1))
    maxAmplitude = 10*np.log10(10**(max(freqDataLog[1:])/10.) - np.mean(10**(freqDataLog[1:]/10.)))
    print('MAX AMPLITUDE ' + str(round(maxAmplitude, 3)) + ' dBm ' )
    maxAmps.append(maxAmplitude)
    if 1:
        plt.figure()
        plt.plot((frequencies/10.**6)[1:], freqDataLog[1:])
        plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
        plt.ylabel(r'Power (dBm)', **label_font)
        plt.show() 

if 1:
    plt.figure()
    #plt.plot((frequencies/10.**6)[1:], freqDataLog[1:])
    #plt.xlabel('Frequency (MHz)', labelpad = 15, **label_font)
    #plt.ylabel(r'Power (dBm)', **label_font)
    #plt.show()
    
    powSig = 10*np.log10(sigAmp**2/50/2*1000)
    plt.plot(np.asarray(freqUncertainties)/sampFreq, maxAmps, 'ro', label = 'TOTAL POWER: ' + str(round(powSig, 3)) + ' dBm \n SAMPLING RATE: ' + str(round(sampFreq/10**9, 2)) + str('GHz'))
    plt.gca().set_xscale('log')
    plt.xlabel('Fractional Sampling Rate Uncertainty', labelpad = 15, **label_font)
    plt.ylabel(r'Power (dBm)', **label_font)
    plt.title('RESOLUTION: ' + str(round(sampFreq/numSamps, 2)) + ' Hz ', **title_font)
    plt.legend(prop = legend_font)
    plt.show()              
    
